# 🎯 Notebook CORRIGIDO: Sem Distorção + Sequência Visual

**Problemas identificados:**
- ❌ Frames distorcidos e deformados
- ❌ Perda de continuidade visual
- ❌ Personagem não mantido consistentemente

**Soluções aplicadas:**
- ✅ Strength reduzido (0.3-0.4) para manter forma original
- ✅ Prompts específicos para manter personagem
- ✅ Negative prompts mais rigorosos
- ✅ Dimensões otimizadas (512x512)
- ✅ Scheduler mais estável


In [1]:
# Instalação
%pip -q install diffusers==0.30.0 transformers accelerate safetensors "imageio[ffmpeg]" pillow



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# 🎯 VERSÃO CORRIGIDA: Homer Simpson sem distorção
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
import time

print("🎯 Iniciando geração CORRIGIDA (sem distorção)...")
start_time = time.time()

# Configuração CORRIGIDA para manter forma original
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512  # ← Tamanho adequado para detalhes
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 8, 12
strength = 0.35  # ← MUITO BAIXO para manter forma original
negative_prompt = "blurry, low quality, distorted, deformed, ugly, disfigured, mutated, extra limbs, missing limbs, bad anatomy, bad proportions, malformed"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames, strength={strength}")

# Carregar imagem do Homer
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Homer carregado: {init_img.size}")

# Pipeline com scheduler estável
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔧 Dispositivo: {device}")

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)

# Scheduler mais estável
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

if device == "cuda":
    pipe.enable_attention_slicing()

print("✅ Pipeline corrigido pronto")

# Prompts específicos para manter Homer
base_prompts = [
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, slight movement",
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, gentle motion",
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, subtle change",
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, natural movement",
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, smooth transition",
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, consistent style",
    "Homer Simpson, yellow skin, cartoon character, maintain original appearance, stable form"
]

frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames (modo corrigido)...")
for i in range(1, frames):
    frame_start = time.time()
    prompt = base_prompts[(i-1) % len(base_prompts)]
    print(f"  Frame {i+1}/{frames}...", end=" ")
    
    out = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=current,
        strength=strength,  # ← MUITO BAIXO para manter forma
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/homer_corrigido.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")
print("🎯 Resultado: Homer mantido sem distorção!")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎯 Iniciando geração CORRIGIDA (sem distorção)...
📊 Config: 512x512, 20 steps, 8 frames, strength=0.35
✅ Homer carregado: (512, 512)
🔧 Dispositivo: cpu


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 25.52it/s]


✅ Pipeline corrigido pronto
🎞️ Gerando 8 frames (modo corrigido)...
  Frame 2/8... 

100%|██████████| 7/7 [00:26<00:00,  3.81s/it]


...

✅ (28.2s)
  Frame 7/8... 

100%|██████████| 7/7 [00:20<00:00,  2.87s/it]


✅ (26.7s)
  Frame 8/8... 

100%|██████████| 7/7 [00:21<00:00,  3.03s/it]


✅ (28.0s)
🎉 CONCLUÍDO em 201.3 segundos!
✅ Vídeo salvo: output/homer_corrigido.mp4
📊 8 frames, 12 FPS, 512x512px
⚡ Tempo médio por frame: 25.2s
🎯 Resultado: Homer mantido sem distorção!


In [3]:
# 🎨 VERSÃO ALTERNATIVA: Interpolação suave entre frames
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, DPMSolverMultistepScheduler
import time

print("🎨 Iniciando geração com INTERPOLAÇÃO SUAVE...")
start_time = time.time()

# Configuração para interpolação suave
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 25, 8.0, 6, 12
strength = 0.25  # ← AINDA MAIS BAIXO para transições suaves
negative_prompt = "blurry, low quality, distorted, deformed, ugly, disfigured, mutated, extra limbs, missing limbs, bad anatomy, bad proportions, malformed, inconsistent, different character"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames, strength={strength}")

# Carregar Homer
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Homer carregado: {init_img.size}")

# Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

if device == "cuda":
    pipe.enable_attention_slicing()

print("✅ Pipeline de interpolação pronto")

# Prompts para movimento suave
movement_prompts = [
    "Homer Simpson, yellow skin, cartoon character, same appearance, very slight head movement",
    "Homer Simpson, yellow skin, cartoon character, same appearance, gentle expression change", 
    "Homer Simpson, yellow skin, cartoon character, same appearance, subtle pose variation",
    "Homer Simpson, yellow skin, cartoon character, same appearance, minimal movement",
    "Homer Simpson, yellow skin, cartoon character, same appearance, consistent character"
]

frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames (interpolação suave)...")
for i in range(1, frames):
    frame_start = time.time()
    prompt = movement_prompts[(i-1) % len(movement_prompts)]
    print(f"  Frame {i+1}/{frames}...", end=" ")
    
    out = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=current,
        strength=strength,  # ← MUITO BAIXO para transições suaves
        num_inference_steps=steps,
        guidance_scale=guidance,
    )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/homer_interpolacao_suave.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")
print("🎨 Resultado: Homer com transições suaves!")


🎨 Iniciando geração com INTERPOLAÇÃO SUAVE...
📊 Config: 512x512, 25 steps, 6 frames, strength=0.25
✅ Homer carregado: (512, 512)


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 36.53it/s]


✅ Pipeline de interpolação pronto
🎞️ Gerando 6 frames (interpolação suave)...
  Frame 2/6... 

100%|██████████| 6/6 [00:17<00:00,  2.84s/it]


✅ (23.9s)
  Frame 3/6... 

...

✅ (24.4s)
  Frame 5/6... 

100%|██████████| 6/6 [00:17<00:00,  2.96s/it]


✅ (24.3s)
  Frame 6/6... 

100%|██████████| 6/6 [00:17<00:00,  2.84s/it]


✅ (23.3s)
🎉 CONCLUÍDO em 123.0 segundos!
✅ Vídeo salvo: output/homer_interpolacao_suave.mp4
📊 6 frames, 12 FPS, 512x512px
⚡ Tempo médio por frame: 20.5s
🎨 Resultado: Homer com transições suaves!
